In [1]:
%pip install -q ipywidgets 
import ipywidgets as widgets
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas
import ipywidgets as widgets

In [2]:
# load the data
data_all = pandas.read_csv("all_runs_spim_pc1.csv",delimiter=",",)

# column headers that are important
inputs = ['meansize','msd','linker_length','linker_conc','temperature','run_index','beta','1/eta','cbeta']
data_all.loc[data_all.linker_conc.isin([0.0]),'linker_conc'] += 0.1 # to include logarithm of zero conc 
data_all['linker_conc'] = np.log10(data_all['linker_conc'])

# select input data and dependent variables
X = data_all[inputs[0:6]].to_numpy() # things to vary
y = data_all[inputs[6:]].to_numpy()  # fit parameters

In [3]:
# determine the color scale of the 2D colormap
init_col_value = 1
final_col_value = 0

# figure fontsize
fontsize = 20

# marker shapes
shapes = ['o','h','s','^']

unique_color_dicts = {input_:np.unique(data_all[input_]) for input_ in inputs[0:6]}
colormap_ticks = {'linker_length':np.array([0,5,10,15]),'linker_conc':np.log10(np.array([0.1,1,10,100])),
                  'temperature':np.arange(20,40,5),'run_index':np.array([1,2,3]),
                  'meansize':np.around(np.linspace(np.min(data_all['meansize']),np.max(data_all['meansize']),4),2),
                  'msd':np.around(np.linspace(np.min(data_all['msd']),np.max(data_all['msd']),4),3)}

shape_dict = {0:'temperature',1:'run_index',2:'linker_conc',3:'linker_length'}
color_dict = {0:'linker_conc',1:'linker_length',2:'meansize',3:'msd',4:'temperature',5:'run_index'}
shape_dict_rev = {v: k for k, v in shape_dict.items()}
color_dict_rev = {v: k for k, v in color_dict.items()}


shape_plot_dict = {'temperature': {20:shapes[0],25:shapes[1],30:shapes[2],35:shapes[3]}, 
                   'run_index': {1:shapes[0],2:shapes[1],3:shapes[2]},
                   'linker_conc':{np.log10(0.1):shapes[0],np.log10(1):shapes[1],np.log10(10):shapes[2],np.log10(100):shapes[3]},
                   'linker_length':{0:shapes[0],5:shapes[1],10:shapes[2],15:shapes[3]}}

variable_unit_dict = {'temperature':r'$\degree C$','run_index':'',
                      'linker_conc':r'','linker_length':r'$bp$',
                      'meansize':r'$um$','msd':r'$um$'}

variable_symbol_dict = {'temperature':r'$T$','run_index':'run index',
                      'linker_conc': r'$log_{10}(C)$','linker_length':r'$l$',
                      'meansize':r'$\mu$','msd':r'$\sigma$'}

In [4]:
out = widgets.Output()

def interactive_plot(shapelab,colorlab0,colorlab1):
    #clear_output(wait=True)
    fig, ax = plt.subplots(1,3,figsize=(18,8),gridspec_kw={'width_ratios':[2,2,0.6]})
    
    ax[0].set_ylabel(r'$\beta$',fontsize=fontsize)
    
    ax[0].set_xlabel(r'$c_{\beta}$',fontsize=fontsize)
    ax[1].set_xlabel(r'$1/\eta$',fontsize=fontsize)
    
    ax[0].set_xscale('log')
    ax[1].set_xscale('log')
    
    ax[0].set_yticks([0.0,0.1,0.2,0.3])
    ax[1].set_yticks([0.0,0.1,0.2,0.3])
    ax[0].tick_params(labelsize=fontsize*3/4)
    ax[1].tick_params(labelsize=fontsize*3/4)
    
    ax[0].set_ylim(-0.01,0.31)
    ax[1].set_ylim(-0.01,0.31)
    
    draw_figure(ax,shapelab,colorlab0,colorlab1)
    plot_shape_legend(ax,shapelab)
    plot_color_legend(ax,colorlab0,colorlab1)
    plt.show()


def choose_color(index,lab1,lab2):
    dat_1 = unique_color_dicts[lab1]
    dat_2 = unique_color_dicts[lab2]
    
    val1 = data_all[lab1][index]
    val2 = data_all[lab2][index]
    
    index1 = np.where(dat_1==val1)[0][0]
    index2 = np.where(dat_2==val2)[0][0]
    
    xx = np.linspace(init_col_value,final_col_value,len(dat_1))
    yy = np.linspace(init_col_value,final_col_value,len(dat_2))
    
    r_val = xx[index1]
    g_val = yy[index2]
    return (r_val,g_val,0)

def choose_shape(index,param):
    return shape_plot_dict[param][data_all[param][index]]

def plot_point0(index,ax,shapelab,colorlab0,colorlab1):
    color = choose_color(index,colorlab0,colorlab1)
    shape = choose_shape(index,shapelab)
    #print("lol1")
    ax[0].scatter(y[index,1],y[index,0],color=color,marker=shape) # beta vs cbeta

def plot_point1(index,ax,shapelab,colorlab0,colorlab1):
    
    color = choose_color(index,colorlab0,colorlab1)
    shape = choose_shape(index,shapelab)
    ax[1].scatter(y[index,2],y[index,0],color=color,marker=shape) # beta vs 1/eta

def draw_figure(ax,shapelab,colorlab0,colorlab1):
    #for index in np.arange(0,np.shape(y)[0]):
    #    plots1[index].remove()
    #    plots2[index].remove()

    
    for index in np.arange(0,np.shape(y)[0]):
        
        plot_point0(index,ax,shapelab,colorlab0,colorlab1)
        
        plot_point1(index,ax,shapelab,colorlab0,colorlab1)
        
def plot_shape_legend(ax,label):  
    # remove existing shape legend
    for line in ax[1].lines + ax[1].collections + ax[1].patches + ax[1].images:
        line.set_label(s='')
        
    # add new shape legend
    sorted_keys = sorted(shape_plot_dict[label].keys())
    for key in sorted_keys:
        markershape = shape_plot_dict[label][key]
        ax[1].scatter([1.0],[2.0],color="black",marker=markershape,s=fontsize*4,
                      label="{}={}{}".format(variable_symbol_dict[label],key,variable_unit_dict[label])) # beta vs 1/eta
  
    ax[1].legend(fontsize=fontsize*3/4)

def plot_color_legend(ax,lab1,lab2):
    
    dat_1 = unique_color_dicts[lab1]
    dat_2 = unique_color_dicts[lab2] 
    
    xx,yy = np.meshgrid(np.linspace(init_col_value,final_col_value,len(dat_1)),
                        np.linspace(init_col_value,final_col_value,len(dat_2)))
    
    X_to_plot = np.dstack((xx, yy, np.zeros_like(xx)))
    
    # adjust color to account for the fact that length=0 is the same for all conc.
    if (lab1 == 'linker_length' and  lab2 == 'linker_conc') or (lab2 == 'linker_length' and  lab1 == 'linker_conc'):
        X_to_plot[0,1:,:] = X_to_plot[1:,0,:] = 1
        
    elif lab1 == lab2:
        for i in range(np.shape(X_to_plot[:,:,0])[0]):
            for j in range(np.shape(X_to_plot[:,:,0])[1]):
                if i != j:
                    X_to_plot[i,j,:] = 1
            
    ax[2].imshow(
        X_to_plot,
        extent = (np.min(dat_1),np.max(dat_1),np.min(dat_2),np.max(dat_2)),
        aspect = 'auto',
        origin = 'lower')
    
    ax[2].set_xticks(colormap_ticks[lab1])
    ax[2].set_yticks(colormap_ticks[lab2])
   
    if variable_unit_dict[lab1] == '':
        ax[2].set_xlabel('{}'.format(variable_symbol_dict[lab1]),fontsize=fontsize)
    else:
        ax[2].set_xlabel('{} ({})'.format(variable_symbol_dict[lab1],variable_unit_dict[lab1]),fontsize=fontsize)
        
    if variable_unit_dict[lab2] == '':
        ax[2].set_ylabel('{}'.format(variable_symbol_dict[lab2]),fontsize=fontsize)
    else:
        ax[2].set_ylabel('{} ({})'.format(variable_symbol_dict[lab2],variable_unit_dict[lab2]),fontsize=fontsize)

In [5]:
widgets.interact(interactive_plot, shapelab = widgets.Dropdown(description = "Shape",options = (shape_dict[0],shape_dict[1],shape_dict[2],shape_dict[3]),disabled=False),
         colorlab0=widgets.Dropdown(description = "Color 1", options = (color_dict[0],color_dict[1],color_dict[2],color_dict[3],color_dict[4],color_dict[5]),
                                     value=color_dict[0],disabled=False),
         colorlab1=widgets.Dropdown(description = "Color 2", options = (color_dict[0],color_dict[1],color_dict[2],color_dict[3],color_dict[4],color_dict[5]),
                                     value=color_dict[1],disabled=False))

interactive(children=(Dropdown(description='Shape', options=('temperature', 'run_index', 'linker_conc', 'linke…

<function __main__.interactive_plot(shapelab, colorlab0, colorlab1)>